# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests, os
from bs4 import BeautifulSoup
import pandas as pd

# II. Instantiate Browser Object 

In [ ]:
url = 'https://ww2.amstat.org/meetings/jsm/2008/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is

# move up to levels to where chromedriver is saved
chrome_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

browser = webdriver.Chrome(executable_path= os.path.join(chrome_path, 'chromedriver'))
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [ ]:
xpath_search = '/html/body/table[2]/tbody/tr[2]/td/form/div/input[6]'
xpath_session = '//tr//td[1]//a[contains(@href, "activity_details")]'
xpath_titles = '//tr//td[2]'
xpath_sponsors = '//tbody//tr//td[3]'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [ ]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [ ]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles) if item.text != "Title"]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor']
session_type = [re.sub(r'(\n|\r|\t)', ' ', item.text) for item in browser.find_elements_by_xpath(xpath_type) 
                if item.text != "Type" ]
session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_session)]


In [ ]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract)

In [ ]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [ ]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [ ]:
df.head(1)

In [ ]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [ ]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2008Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [ ]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

In [ ]:
test = df['Session_URL'].iloc[45]
test

In [ ]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
[item.get('href') for item in soup.find_all('a') if re.search('abstract_details', str(item.get('href')))]

# Run Code to extract workshops and abstract_urls

In [ ]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('abstract_details', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[9].text)
            id_no = soup.find_all('tr')[5].find('strong').text.strip()

            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('abstract_details', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[5].find('strong').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [ ]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

In [ ]:
len(session_url)

In [ ]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

In [ ]:
len(workshop_abstract)

In [ ]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [ ]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [ ]:
df_abstracts.head(1)

In [ ]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [ ]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [ ]:
df_workshops.head(3)

In [ ]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [ ]:
df_abstracts.to_excel(r'{}\2008Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2008Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [ ]:
test = df_abstracts['Abstract_URL'].iloc[0]
test

In [ ]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)

# Code to extract abstract details from abstract URLs

In [ ]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('keyword', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[5].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[14].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [ ]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [ ]:
df_abstracts.to_excel(r'{}\2008Abstracts.xlsx'.format(dest))

In [ ]:
df_abstracts.head(1)

In [ ]:
df_details = df_workshops.append(df_abstracts)

In [ ]:
df.head(0)

In [ ]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [ ]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [ ]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [ ]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [ ]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [ ]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [ ]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [ ]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2008_JSM_data.xlsx'.format(new_dest))